# Generates mesh of 3D Cube 

In [1]:
try
    using Gmsh: gmsh
catch
    using gmsh
end

using GR 

using LinearAlgebra 
using SparseArrays 
using StaticArrays
using StaticRanges

using BenchmarkTools

using Test 

using Plots 

In [4]:
struct Element
    p1::Array{Float64,1}
    p2::Array{Float64,1}
    p3::Array{Float64,1}
    p4::Array{Float64,1}
    n1::Int64
    n2::Int64
    n3::Int64
    n4::Int64
    vol::Float64
end

struct Mesh
    nnodes::Int64
    nelements::Int64 
    # specify one-dimensional array of elements as an array of structs. 
    # we worry about using structArray (if as all) later. 
    Elements::Array{Element,1}
    bndNodeIds::Vector{Int64}
    dofPerElement::Int64       
end 

In [5]:
function volume_tetrahedron(x1::Float64,x2::Float64,x3::Float64,x4::Float64,y1::Float64,y2::Float64,y3::Float64,y4::Float64,z1::Float64,z2::Float64,z3::Float64,z4::Float64)::Float64
    mat = [[x1 y1 z1 1]; [x2 y2 z2 1]; [x3 y3 z3 1]; [x4 y4 z4 1]];
    volume = abs(det(mat))/6;
    return volume
end

volume_tetrahedron (generic function with 1 method)

In [15]:
#generates the mesh for a cube 1*1*1

gmsh.initialize()
model = gmsh.model
model.add("cube_3D")

cl = 1 #mesh size

# Create a new model
gmsh.model.geo.addPoint(0, 0, 0, cl, 1)
gmsh.model.geo.addPoint(1, 0, 0, cl, 2)
gmsh.model.geo.addPoint(1, 1, 0, cl, 3)
gmsh.model.geo.addPoint(0, 1, 0, cl, 4)
gmsh.model.geo.addPoint(0, 0, 1, cl, 5)
gmsh.model.geo.addPoint(1, 0, 1, cl, 6)
gmsh.model.geo.addPoint(1, 1, 1, cl, 7)
gmsh.model.geo.addPoint(0, 1, 1, cl, 8)

gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)
gmsh.model.geo.addLine(5, 6, 5)
gmsh.model.geo.addLine(6, 7, 6)
gmsh.model.geo.addLine(7, 8, 7)
gmsh.model.geo.addLine(8, 5, 8)
gmsh.model.geo.addLine(1, 5, 9)
gmsh.model.geo.addLine(2, 6, 10)
gmsh.model.geo.addLine(3, 7, 11)
gmsh.model.geo.addLine(4, 8, 12)

gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1)
gmsh.model.geo.addCurveLoop([5, 6, 7, 8], 2)
gmsh.model.geo.addCurveLoop([1,10,-5,-9], 3)
gmsh.model.geo.addCurveLoop([-2, 10, 6, -11], 4)
gmsh.model.geo.addCurveLoop([-3, 11, 7, -12], 5)
gmsh.model.geo.addCurveLoop([4, 9, -8, -12], 6)

for i in 1:6
    gmsh.model.geo.addPlaneSurface([i],i)
end

gmsh.model.geo.addSurfaceLoop([1,2,3,4,5,6], 1)
gmsh.model.geo.addVolume([1],1)

# Generate the mesh
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(3)

#..if true, write mesh to file for further processing 
if (true) gmsh.write("cube_3D.msh") end 
#..if true, visualize mesh through the GUI 
if (true) gmsh.fltk.run() end 

gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.000535958s, CPU 0.000453s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 6 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00042125

In [13]:
function genMesh(filename::String)::Mesh
    #the function takes the name of the file as input
    dofPerElement = 12;
    
    gmsh.initialize()
    gmsh.open(filename)
    
    node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
    nnodes = length(node_ids)

    #..sort the node coordinates by ID, such that Node one sits at row 1
    tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end] node_coord[3:3:end]]; 
    sorted = sortslices(tosort , dims = 1);

    node_ids = sorted[:,1]
    xnode = sorted[:,2]
    ynode = sorted[:,3]
    znode = sorted[:,4]

    #get the mesh elements
    element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(3)
    nelements = length(element_ids[1])
    #boundary elements
    bnd_types, bnd_el_ids, bnd_element_connectivity = gmsh.model.mesh.getElements(2)
    
    bnd_node_ids = [];
    #print(bnd_el_ids)
    for i in 1:length(bnd_el_ids[1])
        #retrieve global numbering of the local nodes of the current BOUNDARY element
        bnd_node1_id = bnd_element_connectivity[1][3*(i-1)+1]
        bnd_node2_id = bnd_element_connectivity[1][3*(i-1)+2]
        bnd_node3_id = bnd_element_connectivity[1][3*(i-1)+3]
        if !(bnd_node1_id in bnd_node_ids)
            push!(bnd_node_ids, bnd_node1_id)
        end
        if !(bnd_node2_id in bnd_node_ids)
            push!(bnd_node_ids, bnd_node2_id)
        end
        if !(bnd_node3_id in bnd_node_ids)
            push!(bnd_node_ids, bnd_node3_id)
        end
        
    end
    bnd_node_ids = sort(bnd_node_ids)
    gmsh.finalize()
    
    Elements = Array{Element,1}(undef,nelements);
    total_volume = 0;
    for i in 1:nelements
        
        #....retrieve global numbering of the local nodes of the current element
        node1_id = element_connectivity[1][4*(i-1)+1]
        node2_id = element_connectivity[1][4*(i-1)+2]
        node3_id = element_connectivity[1][4*(i-1)+3]
        node4_id = element_connectivity[1][4*(i-1)+4]
        
        #....retrieve the x and y coordinates of the local nodes of the current element
        xnode1 = xnode[node1_id]; xnode2 = xnode[node2_id]; xnode3 = xnode[node3_id]; xnode4 = xnode[node4_id];
        ynode1 = ynode[node1_id]; ynode2 = ynode[node2_id]; ynode3 = ynode[node3_id]; ynode4 = ynode[node4_id];
        znode1 = znode[node1_id]; znode2 = znode[node2_id]; znode3 = znode[node3_id]; znode4 = znode[node4_id];
        
        #calculate volume of the element and add it to the total volume
        vol = volume_tetrahedron(xnode1, xnode2, xnode3, xnode4, ynode1, ynode2, ynode3, ynode4, znode1, znode2, znode3, znode4)
        total_volume = total_volume + vol;
        #save nodes coordinates, nodes IDs and the volume of one element
        Elements[i] = Element([xnode1; ynode1; znode1],[xnode2; ynode2; znode2],[xnode3; ynode3; znode3],[xnode4; ynode4; znode4], node1_id, node2_id, node3_id, node4_id, vol)
    end
    #print(total_volume) #using cube_3D.msh the volume should be 1
    mesh = Mesh(nnodes, nelements, Elements, bnd_node_ids, dofPerElement)
    return mesh
end

genMesh (generic function with 1 method)

In [14]:
mesh = genMesh("cube_3D.msh")

Info    : Reading 'cube_3D.msh'...
Info    : 27 entities
Info    : 14 nodes
Info    : 68 elements
Info    : Done reading 'cube_3D.msh'


Mesh(14, 24, Element[Element([0.5, 1.0, 0.5], [0.5, 0.5, 1.0], [0.0, 0.5, 0.5], [1.0, 0.5, 0.5], 13, 10, 14, 12, 0.041666666666666664), Element([0.5, 0.5, 0.0], [1.0, 0.5, 0.5], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], 9, 12, 14, 11, 0.041666666666666664), Element([1.0, 0.5, 0.5], [0.5, 0.5, 1.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], 12, 10, 14, 11, 0.041666666666666664), Element([0.0, 0.5, 0.5], [0.5, 1.0, 0.5], [1.0, 0.5, 0.5], [0.5, 0.5, 0.0], 14, 13, 12, 9, 0.041666666666666664), Element([1.0, 0.5, 0.5], [0.5, 1.0, 0.5], [1.0, 1.0, 1.0], [1.0, 1.0, 0.0], 12, 13, 7, 3, 0.041666666666666664), Element([0.5, 0.5, 1.0], [1.0, 0.0, 1.0], [0.0, 0.0, 1.0], [0.5, 0.0, 0.5], 10, 6, 5, 11, 0.041666666666666664), Element([1.0, 0.0, 1.0], [0.5, 0.5, 1.0], [1.0, 1.0, 1.0], [1.0, 0.5, 0.5], 6, 10, 7, 12, 0.041666666666666664), Element([0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.0, 0.0, 1.0], [0.5, 0.0, 0.5], 1, 14, 5, 11, 0.041666666666666664), Element([0.0, 0.0, 1.0], [0.0, 0.5, 0.5], [0.0, 1.0, 1.0], [0.5, 0.5,